# Library

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from keras.utils.np_utils import to_categorical
from keras.models import load_model, Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, SimpleRNN, Dropout

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preprocessing
### Load Data

In [2]:
df = pd.read_pickle('preprocessed_df.pkl')
df = pd.DataFrame(df[['detokenize','akting']])
df.head()

,detokenize,akting
0,the desperate hour lakewood salah cerita suara...,1
1,edisi males review singkat tonton libur dp des...,1
2,plot utama orang deserter pursuit buru wamil j...,1
3,film hereditary horror thrill midsommar gatau ...,0
4,batman manusiawi tarung nya sadis scene pursui...,0


### Vectorize

In [3]:
df['sent_len'] = df['detokenize'].apply(lambda x: len(x.split(" ")))
max_sequence_length = np.round(df['sent_len'].mean() + 2 * df['sent_len'].std()).astype(int)
max_sequence_length

32

In [4]:
MAX_NB_WORDS = 1e5
le = LabelEncoder()

reviews = df["detokenize"]
label = le.fit_transform(df['akting'])
label = np.asarray(to_categorical(label))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(reviews)
review_seq = np.array(tokenizer.texts_to_sequences(reviews), dtype=object)
review_seq = pad_sequences(review_seq, padding='post', maxlen=max_sequence_length)

word_index = tokenizer.word_index
print(f'Dictionary Size\t: {len(word_index)}')
print(f'Example Label\t:\n{label[:5]}')

Dictionary Size	: 17718
Example Label	:
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


# Variables

In [5]:
embedding_dim = 300
vocab_size = min(MAX_NB_WORDS, len(word_index) + 1)
test_size_split = 0.1

# Split Data

In [6]:
x_train, x_test, y_train, y_test = (
    train_test_split(review_seq, 
                     label, 
                     test_size=test_size_split, 
                     stratify = label, 
                     random_state=42)
)

x_train = tf.constant(x_train)
x_test = tf.constant(x_test)

y_train = tf.constant(y_train)
y_test = tf.constant(y_test)

print(f'Train\t| X shape: {x_train.shape}\tY shape: {y_train.shape}')
print(f'Test\t| X shape: {x_test.shape}\tY shape: {y_test.shape}')

Train	| X shape: (15522, 32)	Y shape: (15522, 3)
Test	| X shape: (1725, 32)	Y shape: (1725, 3)


# Model RNN

In [7]:
def RNN(x_train_c, y_train_c, x_test_c, y_test_c):
    rnn = Sequential()
    rnn.add(Embedding(vocab_size, 
                      embedding_dim, 
                      input_length=max_sequence_length,
                      trainable=False))
    rnn.add(SimpleRNN(units=256, activation='relu', recurrent_dropout=0.2, return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=128, activation='relu', return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=64, activation='relu'))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=3, activation='softmax'))
    
    rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience=3, min_delta=0.0001)
    model_checkpoint = ModelCheckpoint('rnn_baseline_akting.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)
    
    history = rnn.fit(x_train_c, y_train_c, epochs= 10, batch_size=128, 
                      validation_data=(x_test_c, y_test_c), verbose = 1,
                      callbacks=[early_stopping, model_checkpoint])
    
    model = load_model('rnn_baseline_akting.h5')
    y_pred = model.predict(x_test_c)
    y_pred_cat = y_pred.argmax(axis=1)
    y_test_cat = np.argmax(y_test_c, axis=1)
   
    cm = confusion_matrix(y_test_cat, y_pred_cat)
    classreport = classification_report(y_test_cat, y_pred_cat)
    f1 = f1_score(y_test_cat, y_pred_cat,  average="macro")
    recall = recall_score(y_test_cat, y_pred_cat, average="macro")
    precision = precision_score(y_test_cat, y_pred_cat, average="macro")
    accuracy = accuracy_score(y_test_cat, y_pred_cat)

    print(classreport)
    print("F1 Score : ", f1)
    print("Precision : ", precision)
    print("Recall : ", recall)
    print("Accuracy : ", accuracy)

    return [f1, precision, recall, accuracy, cm]

# Running

In [8]:
hasil = []

for i in range(1, 6):
    hasil.append([str(i)] + RNN(x_train, y_train, x_test, y_test))

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 0.6234 - accuracy: 0.7952
Epoch 1: val_accuracy improved from -inf to 0.79826, saving model to rnn_baseline_akting.h5
122/122 [==============================] - 19s 127ms/step - loss: 0.6234 - accuracy: 0.7952 - val_loss: 0.6053 - val_accuracy: 0.7983
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 0.6010 - accuracy: 0.7984
Epoch 2: val_accuracy did not improve from 0.79826
122/122 [==============================] - 13s 109ms/step - loss: 0.6010 - accuracy: 0.7984 - val_loss: 0.5895 - val_accuracy: 0.7983
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.5978 - accuracy: 0.7984
Epoch 3: val_accuracy did not improve from 0.79826
122/122 [==============================] - 13s 103ms/step - loss: 0.5978 - accuracy: 0.7984 - val_loss: 0.5947 - val_accuracy: 0.7983
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.5179 - accuracy: 0.8279
Epoch 4: val_accurac

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 0.6284 - accuracy: 0.7964
Epoch 1: val_accuracy improved from -inf to 0.79826, saving model to rnn_baseline_akting.h5
122/122 [==============================] - 19s 126ms/step - loss: 0.6284 - accuracy: 0.7964 - val_loss: 0.5905 - val_accuracy: 0.7983
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 0.5998 - accuracy: 0.7984
Epoch 2: val_accuracy did not improve from 0.79826
122/122 [==============================] - 13s 105ms/step - loss: 0.5998 - accuracy: 0.7984 - val_loss: 0.5920 - val_accuracy: 0.7983
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.5035 - accuracy: 0.8221
Epoch 3: val_accuracy improved from 0.79826 to 0.87826, saving model to rnn_baseline_akting.h5
122/122 [==============================] - 13s 105ms/step - loss: 0.5035 - accuracy: 0.8221 - val_loss: 0.4075 - val_accuracy: 0.8783
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 0.6153 - accuracy: 0.7942
Epoch 1: val_accuracy improved from -inf to 0.79826, saving model to rnn_baseline_akting.h5
122/122 [==============================] - 16s 108ms/step - loss: 0.6153 - accuracy: 0.7942 - val_loss: 0.6011 - val_accuracy: 0.7983
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 0.5871 - accuracy: 0.7984
Epoch 2: val_accuracy did not improve from 0.79826
122/122 [==============================] - 12s 102ms/step - loss: 0.5871 - accuracy: 0.7984 - val_loss: 0.4946 - val_accuracy: 0.7983
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.4673 - accuracy: 0.8225
Epoch 3: val_accuracy improved from 0.79826 to 0.87826, saving model to rnn_baseline_akting.h5
122/122 [==============================] - 13s 103ms/step - loss: 0.4673 - accuracy: 0.8225 - val_loss: 0.3727 - val_accuracy: 0.8783
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 0.6215 - accuracy: 0.7928
Epoch 1: val_accuracy improved from -inf to 0.79826, saving model to rnn_baseline_akting.h5
122/122 [==============================] - 19s 129ms/step - loss: 0.6215 - accuracy: 0.7928 - val_loss: 0.5934 - val_accuracy: 0.7983
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 0.5987 - accuracy: 0.7984
Epoch 2: val_accuracy did not improve from 0.79826
122/122 [==============================] - 13s 107ms/step - loss: 0.5987 - accuracy: 0.7984 - val_loss: 0.5983 - val_accuracy: 0.7983
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.4961 - accuracy: 0.8387
Epoch 3: val_accuracy improved from 0.79826 to 0.85565, saving model to rnn_baseline_akting.h5
122/122 [==============================] - 13s 104ms/step - loss: 0.4961 - accuracy: 0.8387 - val_loss: 0.4503 - val_accuracy: 0.8557
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 0.6180 - accuracy: 0.7951
Epoch 1: val_accuracy improved from -inf to 0.79826, saving model to rnn_baseline_akting.h5
122/122 [==============================] - 21s 147ms/step - loss: 0.6180 - accuracy: 0.7951 - val_loss: 0.5978 - val_accuracy: 0.7983
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 0.6041 - accuracy: 0.7984
Epoch 2: val_accuracy did not improve from 0.79826
122/122 [==============================] - 16s 128ms/step - loss: 0.6041 - accuracy: 0.7984 - val_loss: 0.5905 - val_accuracy: 0.7983
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.5996 - accuracy: 0.7984
Epoch 3: val_accuracy did not improve from 0.79826
122/122 [==============================] - 15s 124ms/step - loss: 0.5996 - accuracy: 0.7984 - val_loss: 0.5893 - val_accuracy: 0.7983
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.6068 - accuracy: 0.7951
Epoch 4: val_accurac

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

# Save Output

In [9]:
df = pd.DataFrame(hasil, columns =['iterasi', 'f1', 'precision', 'recall', 'accuracy', 'cm'])
df.to_excel('rnn_baseline_akting.xlsx')
df.head()

,iterasi,f1,precision,recall,accuracy,cm
0,1,0.556714,0.560093,0.555036,0.892174,"[[0, 35, 22], [0, 1337, 40], [0, 89, 202]]"
1,2,0.563894,0.549481,0.579493,0.894493,"[[0, 27, 30], [0, 1315, 62], [0, 63, 228]]"
2,3,0.562392,0.558789,0.566426,0.895652,"[[0, 32, 25], [0, 1332, 45], [0, 78, 213]]"
3,4,0.548133,0.565774,0.537565,0.889275,"[[0, 36, 21], [0, 1350, 27], [0, 107, 184]]"
4,5,0.295938,0.266087,0.333333,0.798261,"[[0, 57, 0], [0, 1377, 0], [0, 291, 0]]"
